In [59]:
#pewnie nieoptymalnie ale działa XD
input_file_path = 'matikarlowski.json'  # Podmień nazwę pliku wejściowego

def fix_polish_characters(input_file, output_file):
    with open(input_file, 'rb') as file:
        json_data = file.read().decode('utf-8')
        decoded_json = json_data.replace(r'\u00c4\u0099', 'ę').replace(r'\u00c5\u009b','ś').replace(r'\u00c5\u0082','ł')\
            .replace(r'\u00c4\u0087','ć').replace(r'\u00c5\u00bc','ż').replace(r'\u00c4\u0085','ą').replace(r'\u00c3\u00b3','ó')\
            .replace(r'\u00c5\u0084','ń')
        
        with open(output_file, 'w', encoding='utf-8') as output_file:
            output_file.write(decoded_json)

# Przykład użycia
output_file_path = "fixed_"+input_file_path  # Podmień nazwę pliku wyjściowego

fix_polish_characters(input_file_path, output_file_path)


In [60]:
import json

# Wczytaj plik JSON
with open(output_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

    
new_file_name = "data_" + output_file_path.replace(".json", ".txt")

# Otwórz nowy plik tekstowy do zapisu
with open(new_file_name, 'w', encoding='utf-8') as output_file:
    # Iteruj przez wiadomości i zapisuj zawartość do pliku
    for message in data['messages']:
        if 'content' in message:
            output_file.write(message['content'] + '. ')


In [61]:
from transformers import MarianMTModel, MarianTokenizer

def translate_polish_to_english(text):
    # Ładowanie modelu i tokenizera dla tłumaczenia z polskiego na angielski
    model_name = "Helsinki-NLP/opus-mt-pl-en"
    model = MarianMTModel.from_pretrained(model_name)
    tokenizer = MarianTokenizer.from_pretrained(model_name)

    # Tokenizacja tekstu
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)

    # Tłumaczenie
    translation = model.generate(**inputs)
    translated_text = tokenizer.decode(translation[0], skip_special_tokens=True)

    return translated_text


with open(new_file_name, 'r',encoding='utf-8') as file:
    file_content = file.read()

# Przykład użycia
# polish_text = "Przykładowy tekst do przetłumaczenia."
english_translation = translate_polish_to_english(file_content)
print("Tekst po angielsku:", english_translation)


Tekst po angielsku: Thanks a lot. Thanks a lot. I have the same. In your spare time you can confirm. Because I 6 and 2280. And how much did you get?. Thanks. Oh, it's cool. As something I sent her yesterday I sent her an assignment with a regression to an e-mail and wrote from about. Maybe the other things she wants by e-mail. but I don't know. I just thought she wanted an e-mail like she said someone wouldn't happen. She probably wants everything so far by XD e-mail. ok. Because I have so much that I don't need to send anything. It's just for e-mail these tasks that you didn't get on the labs. And I thought she wanted to get to you XD. In Mom's e-mail. Yeah.


In [62]:
from transformers import pipeline

classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)
    
sentences = [english_translation]

model_outputs = classifier(sentences)
print(model_outputs[0])
# produces a list of dicts for each of the labels


[{'label': 'gratitude', 'score': 0.9415872097015381}, {'label': 'curiosity', 'score': 0.10657551139593124}, {'label': 'confusion', 'score': 0.09995421767234802}, {'label': 'neutral', 'score': 0.03301151096820831}, {'label': 'approval', 'score': 0.02062782272696495}, {'label': 'admiration', 'score': 0.014812486246228218}, {'label': 'realization', 'score': 0.011034006252884865}, {'label': 'surprise', 'score': 0.006652976386249065}, {'label': 'annoyance', 'score': 0.006559126544743776}, {'label': 'optimism', 'score': 0.005872928071767092}, {'label': 'excitement', 'score': 0.004444403108209372}, {'label': 'disapproval', 'score': 0.004388326313346624}, {'label': 'joy', 'score': 0.0032444826792925596}, {'label': 'disappointment', 'score': 0.0021386253647506237}, {'label': 'relief', 'score': 0.0020251059904694557}, {'label': 'amusement', 'score': 0.0018855336820706725}, {'label': 'caring', 'score': 0.0018420701380819082}, {'label': 'love', 'score': 0.0013930543791502714}, {'label': 'anger', '